## This example notebook uses Axolotl to fine-tune large foundation models

[Axolotl](https://github.com/OpenAccess-AI-Collective/axolotl) is a tool designed to streamline the fine-tuning of various AI models, offering support for multiple configurations and architectures.

Features:

- Train various Huggingface models such as llama, pythia, falcon, mpt
- Supports fullfinetune, lora, qlora, relora, and gptq
- Customize configurations using a simple yaml file or CLI overwrite
- Load different dataset formats, use custom formats, or bring your own tokenized datasets
- Integrated with xformer, flash attention, rope scaling, and multipacking
- Works with single GPU or multiple GPUs via FSDP or Deepspeed
- Easily run with Docker locally or on the cloud

In [1]:
%pip install -Uq sagemaker
%pip install -Uq datasets
!pip install -Uq transformers==4.33.1 
!pip install -Uq bitsandbytes peft accelerate
!pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import boto3
import sagemaker
import json
from sagemaker import Model, image_uris, serializers, deserializers
from sagemaker.local import LocalSession
import time
from pathlib import Path
from utils import download_model

boto3_session=boto3.session.Session()
# boto3_session=boto3.session.Session()

smr = boto3_session.client("sagemaker-runtime") # sagemaker runtime client for invoking the endpoint
sm = boto3_session.client("sagemaker") 
s3_rsr = boto3_session.resource("s3")
role = sagemaker.get_execution_role()  

sess = sagemaker.session.Session(boto3_session, sagemaker_client=sm, sagemaker_runtime_client=smr)  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
s3_prefix = "llama2-7b-spider"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Download Model

In [2]:
# uncomment to download model
local_model_path = download_model("TheBloke/Llama-2-7B-fp16", "Llama-2-7B-fp16")

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

(…)9903c2ebf7d09970a973ef44d1402239/LICENSE:   0%|          | 0.00/7.02k [00:00<?, ?B/s]

(…)ebf7d09970a973ef44d1402239/USE_POLICY.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

(…)bf7d09970a973ef44d1402239/.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

(…)f44d1402239/pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

(…)bf7d09970a973ef44d1402239/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)70a973ef44d1402239/tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

In [3]:
if list(s3_rsr.Bucket(bucket).objects.filter(Prefix=s3_prefix)) :
    print("Model already exists on the S3 bucket")
    print(f"If you want to upload a new model, please delete the existing model from the S3 bucket with the following command: \n !aws s3 rm --recursive s3://{bucket}/{s3_prefix}")
    s3_model_location = f"s3://{bucket}/{s3_prefix}"
else:
    s3_model_location = sess.upload_data(path=local_model_path.as_posix(), bucket=bucket, key_prefix=s3_prefix)

### Download Data and upload to S3
[Spider dataset with schema](https://huggingface.co/datasets/b-mc2/sql-create-context)

In [4]:
import datasets

# download the training data mhenrichsen/alpaca_2k_test using the HuggingFace datasets library and save output as json
dataset = datasets.load_dataset("b-mc2/sql-create-context")
print(dataset)

data_path = Path("data")
data_path.mkdir(exist_ok=True)

dataset["train"].to_pandas().to_json("data/spider_create_context_train.json", orient="records", lines=True)
s3_data = sess.upload_data(path="data/spider_create_context_train.json", bucket=bucket, key_prefix=f"{s3_prefix}/data")

print(f"Uploaded training data file to {s3_data}")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 78577
    })
})
Uploaded training data file to s3://sagemaker-us-west-2-376678947624/llama2-7b-spider/data/spider_create_context_train.json


In [5]:
!aws s3 ls $s3_data

2023-10-28 13:16:56   19871585 spider_create_context_train.json


In [19]:
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig
import time

str_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

tb_output_config = TensorBoardOutputConfig(s3_output_path=f"s3://{bucket}/{s3_prefix}/tensorboard/{str_time}",
    container_local_output_path="/opt/ml/output/tensorboard")

hyperparameters = {
    "config": "llama2-7b-qlora.yml",
    "deepspeed": "axolotl/deepspeed/zero2.json"
}

# local_sess = LocalSession()
# local_sess.config = {'local': {'local_code': True}}


estimator = PyTorch(
    source_dir = "src",
    entry_point="axolotl/src/axolotl/cli/train.py",
    sagemaker_session=sess,
    role=role,
    instance_count=1, 
    hyperparameters=hyperparameters,
    instance_type="ml.g5.2xlarge", 
    framework_version="2.0.1",
    py_version="py310",
    disable_profiler=True,
    max_run=60*60*24*2,
    keep_alive_period_in_seconds=3600,
    tensorboard_output_config=tb_output_config,
    environment = {"HUGGINGFACE_HUB_CACHE": "/tmp", 
                    "LIBRARY_PATH": "/opt/conda/lib/",
                    "TRANSFORMERS_CACHE": "/tmp",
                    "NCCL_P2P_LEVEL": "NVL"},
    distribution={"torch_distributed": {"enabled": True}} 
)

In [ ]:
estimator.fit({"model": s3_model_location, "train": s3_data})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: pytorch-training-2023-10-28-16-34-30-833


2023-10-28 16:34:31 Starting - Starting the training job...
2023-10-28 16:34:47 Starting - Preparing the instances for training......
2023-10-28 16:35:54 Downloading - Downloading input data...........................................................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-10-28 16:45:39,563 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-10-28 16:45:39,576 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-28 16:45:39,585 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-10-28 16:45:39,588 sagemaker_pytorch_container.training INFO     Invoking TorchDistributed...
2023-10-28 16:45:39,588 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-10-28 16:45:41,243 sagemaker-training-toolkit INFO     Installing dependencies from requireme


2023-10-28 16:45:37 Training - Training image download completed. Training in progress.Collecting hf_transfer (from axolotl==0.3.0->-r requirements.txt (line 6))
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 114.5 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 16.8 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 20.7 MB/s eta 0:00:00
Preparing metadata (setup.py): started
Preparing metadata (setup.py): finished with status 'done'
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 117.1 MB/s eta 0:00:00
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 16.9 MB/s eta 0:00:00
Obtaining dependency information for art from https://files.pythonhosted.org/packages/fc/53/d8792ac2ebb494db0e0ba3ad3f0a9ee71144a5ced266441166f7d038a37e/art-6.1-py3-none-any.whl.metadata
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 20.4 MB/s eta 0:00:00
Obtaining dependency information for fschat==0.2.29 from https://files.pythonhosted.org/packages/a

...

81%|████████  | 582/720 [5:33:49<1:19:19, 34.49s/it]
{'loss': 0.0129, 'learning_rate': 3.9154929577464786e-05, 'epoch': 3.23}
81%|████████  | 583/720 [5:34:22<1:18:18, 34.29s/it]
{'loss': 0.016, 'learning_rate': 3.887323943661972e-05, 'epoch': 3.23}
81%|████████  | 584/720 [5:34:56<1:17:25, 34.16s/it]
{'loss': 0.0097, 'learning_rate': 3.859154929577465e-05, 'epoch': 3.24}
81%|████████▏ | 585/720 [5:35:30<1:16:39, 34.07s/it]
{'loss': 0.0146, 'learning_rate': 3.8309859154929575e-05, 'epoch': 3.25}
81%|████████▏ | 586/720 [5:36:04<1:15:56, 34.00s/it]
{'loss': 0.0102, 'learning_rate': 3.802816901408451e-05, 'epoch': 3.25}
82%|████████▏ | 587/720 [5:36:38<1:15:15, 33.95s/it]
{'loss': 0.0124, 'learning_rate': 3.774647887323944e-05, 'epoch': 3.26}
82%|████████▏ | 588/720 [5:37:12<1:14:37, 33.92s/it]
{'loss': 0.0129, 'learning_rate': 3.746478873239437e-05, 'epoch': 3.26}
82%|████████▏ | 589/720 [5:37:46<1:14:00, 33.89s/it]
{'loss': 0.0136, 'learning_rate': 3.71830985915493e-05, 'epoch': 3.27}


85%|████████▌ | 613/720 [5:51:30<1:07:14, 37.71s/it]
{'loss': 0.0105, 'learning_rate': 3.0422535211267606e-05, 'epoch': 3.4}
85%|████████▌ | 614/720 [5:52:04<1:04:34, 36.55s/it]
{'loss': 0.0113, 'learning_rate': 3.0140845070422537e-05, 'epoch': 3.41}
85%|████████▌ | 615/720 [5:52:38<1:02:32, 35.73s/it]
{'loss': 0.0126, 'learning_rate': 2.9859154929577465e-05, 'epoch': 3.41}
86%|████████▌ | 616/720 [5:53:12<1:00:57, 35.16s/it]
{'loss': 0.011, 'learning_rate': 2.9577464788732395e-05, 'epoch': 3.42}
86%|████████▌ | 617/720 [5:53:46<59:40, 34.76s/it]
{'loss': 0.0139, 'learning_rate': 2.9295774647887326e-05, 'epoch': 3.42}
86%|████████▌ | 618/720 [5:54:20<58:37, 34.49s/it]
{'loss': 0.0103, 'learning_rate': 2.9014084507042254e-05, 'epoch': 3.43}
86%|████████▌ | 619/720 [5:54:53<57:43, 34.29s/it]
{'loss': 0.0137, 'learning_rate': 2.8732394366197185e-05, 'epoch': 3.43}
86%|████████▌ | 620/720 [5:55:27<56:55, 34.15s/it]
{'loss': 0.0145, 'learning_rate': 2.8450704225352116e-05, 'epoch': 3.44}
86

[2023-10-28 23:00:57,045] [INFO] [axolotl.utils.dataloader._len_est:264] [PID:197] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 57782#033
80%|████████  | 4/5 [00:07<00:02,  2.09s/it]
[2023-10-28 23:00:59,638] [INFO] [axolotl.utils.dataloader._len_est:264] [PID:197] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 57782#033
100%|██████████| 5/5 [00:10<00:00,  2.27s/it]
{'eval_loss': 0.018453557044267654, 'eval_runtime': 12.8836, 'eval_samples_per_second': 61.008, 'eval_steps_per_second': 30.504, 'epoch': 3.6}
90%|█████████ | 649/720 [6:12:01<44:36, 37.70s/it]
{'loss': 0.0134, 'learning_rate': 2.028169014084507e-05, 'epoch': 3.6}
90%|█████████ | 650/720 [6:12:35<42:37, 36.54s/it]
{'loss': 0.012, 'learning_rate': 2e-05, 'epoch': 3.61}
90%|█████████ | 651/720 [6:13:09<41:05, 35.73s/it]
{'loss': 0.0124, 'learning_rate': 1.971830985915493e-05, 'epoch': 3.61}
91%|█████████ | 652/720 [6:13:43<39:50, 35.16s/it]
{'loss': 0.0127, 'learning_rate': 1.

[2023-10-28 23:21:28,039] [INFO] [axolotl.utils.dataloader._len_est:264] [PID:197] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 57782#033
80%|████████  | 4/5 [00:07<00:02,  2.09s/it]
[2023-10-28 23:21:30,632] [INFO] [axolotl.utils.dataloader._len_est:264] [PID:197] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 57782#033
100%|██████████| 5/5 [00:10<00:00,  2.27s/it]
{'eval_loss': 0.01807444542646408, 'eval_runtime': 12.8845, 'eval_samples_per_second': 61.003, 'eval_steps_per_second': 30.502, 'epoch': 3.79}
95%|█████████▌| 685/720 [6:32:32<21:59, 37.70s/it]
{'loss': 0.0135, 'learning_rate': 1.0140845070422535e-05, 'epoch': 3.8}
95%|█████████▌| 686/720 [6:33:06<20:42, 36.54s/it]
{'loss': 0.0126, 'learning_rate': 9.859154929577465e-06, 'epoch': 3.81}
95%|█████████▌| 687/720 [6:33:40<19:39, 35.73s/it]
{'loss': 0.0133, 'learning_rate': 9.577464788732394e-06, 'epoch': 3.81}
96%|█████████▌| 688/720 [6:34:14<18:45, 35.16s/it]
{'loss': 0.0143, '

[2023-10-28 23:41:58,950] [INFO] [axolotl.utils.dataloader._len_est:264] [PID:197] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 57782#033
80%|████████  | 4/5 [00:07<00:02,  2.09s/it]
[2023-10-28 23:42:01,543] [INFO] [axolotl.utils.dataloader._len_est:264] [PID:197] [RANK:0] packing_efficiency_estimate: 1.0 total_num_tokens per device: 57782#033
100%|██████████| 5/5 [00:10<00:00,  2.27s/it]
{'eval_loss': 0.017803680151700974, 'eval_runtime': 12.8854, 'eval_samples_per_second': 60.999, 'eval_steps_per_second': 30.5, 'epoch': 3.99}
100%|██████████| 5/5 [00:10<00:00,  2.27s/it]
{'train_runtime': 24786.4228, 'train_samples_per_second': 12.554, 'train_steps_per_second': 0.029, 'train_loss': 0.03561719365987099, 'epoch': 3.99}
100%|██████████| 720/720 [6:53:06<00:00, 34.43s/it]
[2023-10-28 23:42:38,071] [INFO] [axolotl.train.train:120] [PID:197] [RANK:0] Training Completed!!! Saving pre-trained model to /opt/ml/model#033
2023-10-28 23:42:42,302 sagemaker-training-too

## Check Tensorboard report

In [26]:
f"s3://{bucket}/{s3_prefix}/tensorboard/{str_time}"

's3://sagemaker-us-west-2-376678947624/llama2-7b-spider/tensorboard/2023-10-28-16-34-30'

## Test model performance before and after fine tuning

In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

### Before fine tuning

In [30]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
SELECT * FROM table_name_12 WHERE frequency_mhz > 91.5 AND city_of_license = 'hyannis, nebraska'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz


### After fine tuning

In [33]:
lora_path = estimator.model_data
# lora_path = "s3://sagemaker-us-west-2-376678947624/pytorch-training-2023-10-28-16-34-30-833/output/model.tar.gz"

In [34]:
!aws s3 cp {lora_path} .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
download: s3://sagemaker-us-west-2-376678947624/pytorch-training-2023-10-28-16-34-30-833/output/model.tar.gz to ./model.tar.gz


In [35]:
!mkdir -p lora

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
!tar -xzf model.tar.gz -C lora

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'


In [37]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "lora")

In [38]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
 SELECT class FROM table_name_12 WHERE frequency_mhz > 91.5 AND city_of_license = "hyannis, nebraska"
